#### Import packages

In [1]:
import ee
ee.Initialize()
import geemap
Map = geemap.Map()

#### Load collections

In [46]:
# forest mask from ESA CCI (unchanged forests)
forestmask = ee.Image('users/marcogirardello/phenoutils/mask_unchanged_500m')
# entire collection from Sentinel 2
collectionS2 = ee.ImageCollection("COPERNICUS/S2_SR")
# sampling areas
samples = ee.FeatureCollection('users/marcogirardello/exploratoryproject/samples')

In [74]:
f = collectionS2.filterBounds(onesq).first().select('B4')

In [78]:
f.projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32719',
 'transform': [10, 0, 600000, 0, -10, 9100000]}

In [76]:
f1.nominalScale().getInfo()

10

In [22]:
def maskCloudAndShadows(image):
    cloudProb = image.select('MSK_CLDPRB')
    snowProb = image.select('MSK_SNWPRB')
    cloud = cloudProb.lt(5)
    snow = snowProb.lt(5)
    scl = image.select('SCL')
    shadow = scl.eq(3) # 3 = cloud shadow
    cirrus = scl.eq(10) # 10 = cirrus
    # cloud probability less than 5% or cloud shadow classification
    mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
    return image.updateMask(mask)    

In [28]:
onesq = samples.filterMetadata('polyID','equals',1)
Map.addLayer(onesq)

#### Filter collection and add NDVI

In [36]:
res = collectionS2.\
    filterBounds(onesq).\
    filterDate('2017-03-28','2020-12-31').filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',30).\
    map(maskCloudAndShadows).\
    map(lambda image: image.normalizedDifference(['B8', 'B4']).rename('NDVI')).select('NDVI')

In [79]:
img = res.mean()
img1 = img.setDefaultProjection(crs = 'EPSG:32719',scale = 10).reduceResolution(ee.Reducer.stdDev(), bestEffort = False, maxPixels = 200).reproject(ee.Projection('EPSG:32719').scale(100, 100)).updateMask(1) 

In [86]:
Map.addLayer(onesq)

In [84]:
Map.addLayer(img1,'','stdvv')

In [88]:
task = ee.batch.Export.image.toAsset(image = img1,description = 's2test',
                                     assetId='users/marcogirardello/exploratoryproject/s2test',
                                     crs = 'EPSG:32719',scale = 10,maxPixels=1e13,region=onesq.geometry())
task.start()

In [ ]:
img.reduceRegion()

In [82]:
#Map = geemap.Map()
Map.set_center(-67.4484,-7.9461,9)
Map


Map(center=[-7.9461, -67.4484], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…

#### Check temporal coverage

In [18]:
# calculate total number of observations and filter by forest cover
countS2 = s2_2019.select(0).count().updateMask(forestmask.eq(1))

In [14]:
#Map.addLayer(countS2,{'min':1,'max':200,'palette':['blue','green','yellow','red']},'S2 count filtered')

In [19]:
#Map

#### Aggregate data 16-day 